### Аналитикам: большая шпаргалка по Pandas

[Алексей Куличевский](https://smysl.io/blog/pandas/)

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BA%D0%B5%D0%B9%D1%81%D1%8B%20%D0%BF%D0%BE%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D1%83%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/0.%20%D0%9C%D0%B0%D1%80%D0%BA%D0%B5%D1%82%D0%B8%D0%BD%D0%B3.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

Код на чистом Python:

```Python
with open('file.csv') as f:
    content = f.readlines()
    content = [x.split(',').replace('\n','') for x in content]
```

Код, написанный с помощью pandas:

```Python
data = pd.read_csv('file.csv')
```
Pandas добавляет в Python новые структуры данных — серии и датафреймы.

### Структуры данных: серии и датафреймы

Серии — одномерные массивы данных. Они очень похожи на списки, но отличаются по поведению — например, операции применяются к списку целиком, а в сериях — поэлементно.

То есть, если список умножить на 2, получите тот же список, повторенный 2 раза.

In [ ]:
# чистый Python
vector = [1, 2, 3]
vector * 2

А если умножить серию, ее длина не изменится, а вот элементы удвоятся.

In [ ]:
import pandas as pd
series = pd.Series([1, 2, 3])
series

In [ ]:
series * 2

Обратите внимание на первый столбик вывода. Это индекс, в котором хранятся адреса каждого элемента серии. Каждый элемент потом можно получать, обратившись по нужному адресу.

In [ ]:
series = pd.Series(['foo', 'bar'])
series

In [ ]:
series[0]

Еще одно отличие серий от списков — в качестве индексов можно использовать произвольные значения (похоже на словари), это делает данные нагляднее. Представим, что мы анализируем помесячные продажи. Используем в качестве индексов названия месяцев, значениями будет выручка:

In [ ]:
months = ['jan', 'feb', 'mar', 'apr']
sales = [100, 200, 300, 400]

data = pd.Series(data=sales, index=months)
data

Теперь можем получать значения каждого месяца:

In [ ]:
data['feb']

Так как серии — одномерный массив данных, в них удобно хранить измерения по одному. На практике удобнее группировать данные вместе. Например, если мы анализируем помесячные продажи, полезно видеть не только выручку, но и количество проданных товаров, количество новых клиентов и средний чек. Для этого отлично подходят датафреймы.

Датафреймы — это таблицы. У них есть строки, колонки и ячейки.

Технически, колонки датафреймов — это серии. Поскольку в колонках обычно описывают одни и те же объекты, то все колонки делят один и тот же индекс:

In [ ]:
months = ['jan', 'feb', 'mar', 'apr']

# словарь:
sales = {
    'revenue':     [100, 200, 300, 400], # выручка
    'items_sold':  [23, 43, 55, 65], # количество проданных товаров
    'new_clients': [10, 20, 30, 40] # количество новых клиентов
}

sales_df = pd.DataFrame(data=sales, index=months)
sales_df

Бывает, что мы не знаем, что собой представляют данные, и не можем задать структуру заранее. Тогда удобно создать пустой датафрейм и позже наполнить его данными.

In [ ]:
df = pd.DataFrame()

А иногда данные уже есть, но хранятся в переменной из стандартного Python, например, в словаре. Чтобы получить датафрейм, эту переменную передаем в ту же команду:

In [ ]:
df = pd.DataFrame(data=sales, index=months)

Случается, что в некоторых записях не хватает данных. Например, посмотрите на список `goods_sold` — в нём продажи, разбитые по товарным категориям. 

За первый месяц мы продали машины, компьютеры и программное обеспечение. Во втором машин нет, зато появились велосипеды, а в третьем снова появились машины, но велосипеды исчезли:

In [ ]:
goods_sold = [
    {'computers': 10, 'cars': 1, 'soft': 3},
    {'computers': 4, 'soft': 5, 'bicycles': 1},
    {'computers': 6, 'cars': 2, 'soft': 3}
]

Если загрузить данные в датафрейм, Pandas создаст колонки для всех товарных категорий и, где это возможно, заполнит их данными:

In [ ]:
pd.DataFrame(goods_sold)

Обратите внимание, продажи велосипедов в первом и третьем месяце равны `NaN` — расшифровывается как `Not a Number`. Так Pandas помечает отсутствующие значения.

Теперь разберем, как загружать данные из файлов. Чаще всего данные хранятся в экселевских таблицах или csv-, tsv- файлах.

Экселевские таблицы читаются с помощью команды `pd.read_excel()`. Параметрами нужно передать адрес файла на компьютере и название листа, который нужно прочитать. Команда работает как с xls, так и с xlsx:

```Python
pd.read_excel('file.xlsx', sheet_name='Sheet1')
```

Файлы формата csv и tsv — это текстовые файлы, в которых данные отделены друг от друга запятыми или табуляцией:

```
# CSV
month,customers,sales
feb,10,200

# TSV
month\tcustomers\tsales
feb\t10\t200
```

Оба читаются с помощью команды `.read_csv()`, символ табуляции передается параметром `sep` (от англ. separator — разделитель):

```Python
pd.read_csv('file.csv')
pd.read_csv('file.tsv', sep='\t')
```

При загрузке можно назначить столбец, который будет индексом. Представьте, что мы загружаем таблицу с заказами. У каждого заказа есть свой уникальный номер, Если назначим этот номер индексом, сможем выгружать данные командой `df[order_id]`. Иначе придется писать фильтр `df[df["id"] == order_id ]`.

Чтобы назначить колонку индексом, добавим в команду `read_csv()` параметр `index_col`, равный названию нужной колонки:

```Python
pd.read_csv('file.csv', index_col='id')
```

После загрузки данных в датафрейм, хорошо бы их исследовать — особенно, если они вам незнакомы.

### Исследуем загруженные данные

Представим, что мы анализируем продажи американского интернет-магазина. У нас есть данные о заказах и клиентах. Загрузим файл с продажами интернет-магазина в переменную `orders`. Раз загружаем заказы, укажем, что колонка `id` пойдет в индекс:

In [ ]:
url = 'https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/orders.csv'

orders = pd.read_csv(url, index_col='id')

Расскажу о четырех атрибутах, которые есть у любого датафрейма: `.shape`, `.columns`, `.index` и `.dtypes`.

`.shape` показывает, сколько в датафрейме строк и колонок. Он возвращает пару значений `(n_rows, n_columns)`. Сначала идут строки, потом колонки.

In [ ]:
orders.shape

В датафрейме 5009 строк и 4 колонки.

Окей, масштаб оценили. Теперь посмотрим, какая информация содержится в каждой колонке. С помощью `.columns` узнаем названия колонок:

In [ ]:
orders.columns

Теперь видим, что в таблице есть дата заказа, метод доставки, номер клиента и выручка.

С помощью `.dtypes` узнаем типы данных, находящихся в каждой колонке и поймем, надо ли их обрабатывать. Бывает, что числа загружаются в виде текста. Если мы попробуем сложить две текстовых значения `'1' + '1'`, то получим не число `2`, а строку `'11'`:

In [ ]:
orders.dtypes

Тип `object` — это текст, `float64` — это дробное число типа `3,14`.

C помощью атрибута `.index` посмотрим, как называются строки:

In [ ]:
orders.index

Ожидаемо, в индексе датафрейма номера заказов: `100762`, `100860` и так далее.

В колонке `sales` хранится стоимость каждого проданного товара. Чтобы узнать разброс значений, среднюю стоимость и медиану, используем метод `.describe()`:

In [ ]:
orders.describe()

Наконец, чтобы посмотреть на несколько примеров записей датафрейма, используем команды `.head()` и `.sample()`. Первая возвращает 5 записей из начала датафрейма. Вторая — 5 случайных записей:

In [ ]:
orders.head()

In [ ]:
orders.sample(5)

Получив первое представление о датафреймах, теперь обсудим, как доставать из него данные.

### Получаем данные из датафреймов

Данные из датафреймов можно получать по-разному: указав номера колонок и строк, использовав условные операторы или язык запросов.

Продолжаем анализировать продажи интернет-магазина, которые загрузили в предыдущем разделе. Допустим, я хочу вывести столбец `sales`. Для этого название столбца нужно заключить в квадратные скобки и поставить после них названия датафрейма: `orders['sales']`:

In [ ]:
orders['sales']

Обратите внимание, результат команды — новый датафрейм с таким же индексом.

Если нужно вывести несколько столбцов, в квадратные скобки нужно вставить список с их названиями: `orders[['customer_id', 'sales']]`. 

Будьте внимательны: квадратные скобки стали двойными. Первые — от датафрейма, вторые — от списка:

In [ ]:
orders[['customer_id', 'sales']]

Перейдем к строкам. Их можно фильтровать по индексу и по порядку. Например, мы хотим вывести только заказы `100363`, `100391` и `100706`, для этого есть команда `.loc[]`:

In [ ]:
show_these_orders = [100363, 100363, 100706]

In [ ]:
orders.loc[show_these_orders]

А в другой раз бывает нужно достать просто заказы с `1` по `3` по порядку, вне зависимости от их номеров в таблицемы. Тогда используют команду `.iloc[]`:

In [ ]:
show_these_orders = [1, 2, 3]
orders.iloc[show_these_orders]

Можно фильтровать датафреймы по колонкам и столбцам одновременно:

In [ ]:
columns = ['customer_id', 'sales']
rows = [100363, 100363, 100706]
orders.loc[rows][columns]

Часто вы не знаете заранее номеров заказов, которые вам нужны. Например, если задача — получить заказы, стоимостью более `1000` рублей. Эту задачу удобно решать с помощью условных операторов.

### Задача

Необходимо узнать, откуда приходят самые большие заказы. Начнем с того, что достанем все покупки стоимостью более 1000 долларов:

In [ ]:
filter_large = orders['sales'] > 1000

In [ ]:
orders.loc[filter_large]

Операция `orders['sales'] > 1000` идет по каждому элементу серии и, если условие выполняется, возвращает `True`. Если не выполняется — `False`. Получившуюся серию мы сохраняем в переменную `filter_large`.

Вторая команда фильтрует строки датафрейма с помощью серии. Если элемент `filter_large` равен `True`, заказ отобразится, если `False` — нет. Результат — датафрейм с заказами, стоимостью более `1000` долларов.

In [ ]:
# можно без .loc и в одной строчке:
orders[orders['sales'] > 1000]

Интересно, сколько дорогих заказов было доставлено первым классом? Добавим в фильтр ещё одно условие:

In [ ]:
filter_large = orders['sales'] > 1000
filter_first_class = (orders['ship_mode'] == 'First')

orders[filter_large & filter_first_class]

Логика не изменилась. В переменную `filter_large` сохранили серию, удовлетворяющую условию `orders['sales'] > 1000`. В `filter_first_class` — серию, удовлетворяющую `orders['ship_mode'] == 'First'`.

Затем объединили обе серии с помощью логического `И`: `filter_first_class & filter_first_class`. Получили новую серию той же длины, в элементах которой `True` только у заказов, стоимостью больше `1000`, доставленных первым классом. Таких условий может быть сколько угодно.

Еще один способ решить предыдущую задачу — использовать язык запросов. 

Все условия пишем одной строкой `'sales > 1000 & ship_mode == 'First'` и передаем ее в метод `.query()`. Запрос получается компактнее.

In [ ]:
orders.query("sales > 1000 and ship_mode=='First'")

Значения для фильтров можно сохранить в переменной, а в запросе сослаться на нее с помощью символа `@`: `sales > @sales_filter`.

In [ ]:
sales_filter = 1000
ship_mode_filter = 'First'
orders.query('sales > @sales_filter & ship_mode > @ship_mode_filter')

Разобравшись, как получать куски данных из датафрейма, перейдем к тому, как считать агрегированные метрики: количество заказов, суммарную выручку, средний чек, конверсию.

### Считаем производные метрики

Задача: посчитаем, сколько денег магазин заработал с помощью каждого класса доставки. Начнем с простого — просуммируем выручку со всех заказов. Для этого используем метод `.sum()`:

In [ ]:
orders['sales'].sum()

Добавим класс доставки. Перед суммированием сгруппируем данные с помощью метода `.groupby()`:

In [ ]:
orders.groupby('ship_mode')['sales'].sum()

`3.514284e+05` — научный формат вывода чисел. Означает $3.51 * 10^5$. Нам такая точность не нужна, поэтому можем сказать Pandas, чтобы округлял значения до сотых:

In [ ]:
pd.options.display.float_format = '{:,.1f}'.format

In [ ]:
orders.groupby('ship_mode')['sales'].sum()

Другое дело. Теперь видим сумму выручки по каждому классу доставки. По суммарной выручке неясно, становится лучше или хуже. Добавим разбивку по датам заказа:

In [ ]:
orders.groupby(['ship_mode', 'order_date'])['sales'].sum()

Видно, что выручка прыгает ото дня ко дню: иногда `10` долларов, а иногда `378`. 

Интересно, это меняется количество заказов или средний чек? 

Добавим к выборке количество заказов. Для этого вместо `.sum()` используем метод `.agg()`, в который передадим список с названиями нужных функций.

In [ ]:
orders.groupby(['ship_mode', 'order_date'])['sales'].agg(['sum', 'count'])

Ого, получается, что это так прыгает средний чек. Интересно, а какой был самый удачный день? Чтобы узнать, отсортируем получившийся датафрейм: выведем 10 самых денежных дней по выручке:

In [ ]:
orders.groupby(['ship_mode', 'order_date'])['sales'].agg(['sum']).sort_values(by='sum', ascending=False).head(10)

Параметр `ascending` указывает тип сортировки: по возрастанию или по убыванию.

Команда разрослась, и её теперь неудобно читать. Чтобы упростить, можно разбить её на несколько строк. В конце каждой строки ставим обратный слеш \\:

In [ ]:
orders \
    .groupby(['ship_mode', 'order_date'])['sales'] \
    .agg(['sum']) \
    .sort_values(by='sum', ascending=False) \
    .head(10)

В самый удачный день — `18 марта 2014 года` — магазин заработал `27 тысяч долларов с помощью стандартного класса доставки`. 

Интересно, откуда были клиенты, сделавшие эти заказы? Чтобы узнать, надо объединить данные о заказах с данными о клиентах.

### Объединяем несколько датафреймов

До сих пор мы смотрели только на таблицу с заказами. Но ведь у нас есть еще данные о клиентах интернет-магазина. Загрузим их в переменную `customers` и посмотрим, что они собой представляют:

In [ ]:
url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/customers.csv"

customers = pd.read_csv(url, index_col='id')

In [ ]:
customers.head()

Мы знаем тип клиента, место его проживания, его имя и имя контактного лица. У каждого клиента есть уникальный номер `id`. Этот же номер лежит в колонке `customer_id` таблицы `orders`. Значит мы можем найти, какие заказы сделал каждый клиент. Например, посмотрим, заказы пользователя `CG-12520`:

In [ ]:
cust_filter = 'CG-12520'
orders.query('customer_id == @cust_filter')

Вернемся к задаче из предыдущего раздела: узнать, что за клиенты, которые сделали `18 марта заказы со стандартной доставкой`. Для этого объединим таблицы с клиентами и заказами. 

Датафреймы объединяют с помощью методов `.concat()`, `.merge()` и `.join()`. Все они делают одно и то же, но отличаются синтаксисом — на практике достаточно уметь пользоваться одним из них.

Покажу на примере `.merge()`:

In [ ]:
new_df = pd.merge(orders, customers, 
                  how='inner', 
                  left_on='customer_id', # номер клиента
                  right_index=True) # номер клиента в индексе

In [ ]:
new_df.columns

In [ ]:
new_df.head()

В `.merge()` я сначала указал названия датафреймов, которые хочу объединить. 
Затем уточнил, как именно их объединить и какие колонки использовать в качестве ключа.

Ключ — это колонка, связывающая оба датафрейма. В нашем случае — номер клиента. В таблице с заказами он в колонке `customer_id`, а таблице с клиентами — в индексе. Поэтому в команде мы пишем: `left_on='customer_id', right_index=True`.

### Решаем задачу

Найдем `5 городов, принесших самую большую выручку в 2016 году`.

Для начала отфильтруем заказы из `2016 года`:

In [ ]:
orders_2016 = orders.query("order_date >= '2016-01-01' & order_date <= '2016-12-31'")

In [ ]:
orders_2016.head()

Город — это атрибут пользователей, а не заказов. Добавим информацию о пользователях:

In [ ]:
with_customers_2016 = pd.merge(customers, orders_2016, 
                               how='inner', 
                               left_index=True, 
                               right_on='customer_id')

Cруппируем получившийся датафрейм по городам и посчитаем выручку:

In [ ]:
grouped_2016 = with_customers_2016.groupby('city')['sales'].sum()

In [ ]:
grouped_2016.head()

Отсортируем по убыванию продаж и оставим топ-5:

In [ ]:
top5 = grouped_2016.sort_values(ascending=False).head(5)

In [ ]:
top5

### Упражнение:

1. Сколько заказов, отправлено первым классом за последние 5 лет?
2. Сколько в базе клиентов из Калифорнии?
3. Сколько заказов они сделали?
4. Постройте сводную таблицу средних чеков по всем штатам за каждый год.